In [7]:
from PyPDF2 import PdfReader
directory = "/home/shreya/sem6/nlp/papers/qa_over_large_struct_doc.pdf"

In [8]:
from typing_extensions import Concatenate

In [10]:
def load_docs(filename):
    pdfreader = PdfReader(directory)
    # read text from pdf
    raw_text = ''
    for i, page in enumerate(pdfreader.pages):
        content = page.extract_text()
        if content:
            raw_text += content
            
    return raw_text

In [11]:
text = load_docs(directory)

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [18]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_text(documents) #to use across multiple documents rather than a string, use split_documents
    return docs

docs = split_docs(text)
print(len(docs))

43


In [21]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [35]:
print(len(embeddings.embed_query("Hello World")))

4096


In [29]:
import pinecone
from langchain.vectorstores import Pinecone

In [37]:
pinecone.init(
    api_key="539e3f4e-1c1f-4894-a175-4d2abd5fbd31",
    environment="gcp-starter"
)

index_name = "langchain-pdfqna"

index = Pinecone.from_texts(texts = docs, embedding=embeddings, index_name = index_name)


In [41]:
def get_similiar_docs(query, k=2, score=False):
    if score:
        similar_docs = index.similarity_search_with_score(query, k=k)
    else:
        similar_docs = index.similarity_search(query, k=k)
    print(similar_docs)
    
    return similar_docs

In [20]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [39]:
from langchain.chains.question_answering import load_qa_chain

In [40]:
chain = load_qa_chain(llm, chain_type="stuff")

In [42]:
def get_answer(query):
    similar_docs = get_similiar_docs(query)
    answer = chain.run(input_documents=similar_docs, question=query)
    return answer

In [43]:
query = "What is PDF question answering"
answer = get_answer(query)
print(answer)

[Document(page_content='the questions are primarily focused on document\ncontents. The PDFTriage evaluation dataset seeksto expand on the question types in these datasets,\ngetting questions that can reference the document\nstructure or content, can be extractive or abstractive,\nand can require long-form answers or rewrites.\n3 PDFTriage: Structured Retrieval from\nDocument Metadata\nThePDFTriage approach consists of three steps to\nanswer a user’s question, shown in Figure 1:\n1.Generate document metadata (Sec. 3.1):\nExtract the structural elements of a document\nand convert them into readable metadata.\n2.LLM-based triage (Sec. 3.2): Query the\nLLM to select the precise content (pages, sec-\ntions, retrieved content) from the document.\n3.Answer using retrieved content (Sec. 3.3):\nBased on the question and retrieved content,\ngenerate an answer.# of Documents 82\n# of Questions 908\nEasy Questions 393\nMedium Questions 144\nHard Questions 266\n“Unsure” Questions 105\nTable 1: Data